# We could expect that the mean number of crashes over the weekends is bigger than during labour days. Make a two sample t-test to evaluate this hypothesis

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

In [3]:
df_clean= pd.read_csv('..\data\clean_crash_data.csv')
display(df_clean.head())

,report_number,local_case_number,agency_name,acrs_report_type,crash_date/time,hit/run,lane_number,number_of_lanes,road_grade,nontraffic,at_fault,collision_type,weather,light,road_condition,latitude,longitude,location
0,mcp3048005t,190046316,montgomery,property damage crash,2019-09-27 09:38:00,no,0,0,level,yes,driver,single vehicle,clear,daylight,foreign material,39.026700,-77.136785,"(39.0267, -77.136785)"
1,dm8376000z,210002024,takoma,property damage crash,2021-01-16 21:24:00,no,0,2,level,no,driver,same direction sideswipe,clear,dark lights on,no defects,38.979030,-77.015981,"(38.97903039, -77.01598148)"
2,mcp12050078,210038391,montgomery,property damage crash,2021-09-27 15:00:00,yes,0,0,grade downhill,yes,driver,straight movement angle,clear,daylight,holes ruts etc,39.023680,-77.145418,"(39.02367954, -77.14541849)"
3,mcp003600g5,220000831,montgomery,property damage crash,2022-01-07 19:00:00,no,0,0,level,yes,unknown,single vehicle,clear,dark lights on,holes ruts etc,39.058047,-77.125307,"(39.058047, -77.125307)"
4,ej78460055,200017938,gaithersburg,property damage crash,2020-05-02 14:16:00,yes,0,0,level,yes,driver,straight movement angle,clear,daylight,holes ruts etc,39.147956,-77.229616,"(39.14795617, -77.2296164)"


In [4]:
from scipy.stats import ttest_ind
df_hypothesis = df_clean.copy()

df_hypothesis['crash_date/time'] = pd.to_datetime(df_hypothesis['crash_date/time'])
df_hypothesis['day_of_week'] = df_hypothesis['crash_date/time'].dt.dayofweek
df_hypothesis['is_weekend'] = (df_hypothesis['day_of_week'] >= 5).astype(int)

crash_counts = df_hypothesis.groupby('is_weekend')['report_number'].count()

# Separate data for weekends and weekdays
weekend_counts = df_hypothesis[df_hypothesis['is_weekend'] == 1]['number_of_lanes']
weekday_counts = df_hypothesis[df_hypothesis['is_weekend'] == 0]['number_of_lanes']

# Perform the two-sample t-test
stat, p_val = ttest_ind(weekend_counts, weekday_counts, alternative='less')

# Output the results
print(f'T-statistic: {stat:.3f}')
print(f'P-value: {p_val:.3f}')

critical_value = scipy.stats.t.ppf(0.05, df=len(df_hypothesis)-1)
print("The critical value is {:.3f}".format(critical_value))


T-statistic: -6.419
P-value: 0.000
The critical value is -1.645


P-values is less than alpha , therefore we reject the null hypothesis

In [8]:
plt.figure(figsize=(10, 6))

# ECDF plot for weekends
ecdf_weekend = ECDF(weekend_counts)
plt.step(ecdf_weekend.x, ecdf_weekend.y, label='Weekends', linestyle='-', where='post')

# ECDF plot for weekdays
ecdf_weekday = ECDF(weekday_counts)
plt.step(ecdf_weekday.x, ecdf_weekday.y, label='Weekdays', linestyle='-', where='post')

# Mark the critical value on the plot
critical_value = scipy.stats.t.ppf(0.05, df=len(df_hypothesis)-1)
plt.axvline(x=critical_value, color='r', linestyle='--', label='Critical Value')

plt.title('ECDF of number_of_lanes for Weekends and Weekdays')
plt.xlabel('Number of Lanes')
plt.ylabel('Cumulative Probability')
plt.legend()
plt.show()

NameError: name 'ECDF' is not defined

<Figure size 1000x600 with 0 Axes>